In [ ]:
# justWatch

In [60]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import time
import pandas as pd

# 크롬 드라이버 설정 및 페이지 열기
driver = webdriver.Chrome()
driver.get('https://www.justwatch.com/kr/%EB%8F%99%EC%98%81%EC%83%81%EC%84%9C%EB%B9%84%EC%8A%A4/netflix/TV-%ED%94%84%EB%A1%9C%EA%B7%B8%EB%9E%A8?genres=drm')

# 페이지 로딩 대기 및 스크롤 끝까지 내리기
try:
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.title-list')))

    SCROLL_PAUSE_TIME = 2
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # 페이지 소스 가져오기
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    
    # 각 드라마의 링크 추출
    drama_links = [a['href'] for a in soup.select('a.title-list-grid__item--link')]
    
    drama_data = []
    
    for link in drama_links:
        driver.get(f"https://www.justwatch.com{link}")
        time.sleep(2)  # 페이지 로딩 대기
        
        detail_page = BeautifulSoup(driver.page_source, 'html.parser')
        
        title = detail_page.select_one('div.title-block h1').text.strip() if detail_page.select_one('div.title-block h1') else 'N/A'
        original_title = detail_page.select_one('div.title-block > h3').text.strip() if detail_page.select_one('div.title-block > h3') else 'N/A'
        year = detail_page.select_one('div.title-block__container > div.title-block > div > span').text.strip() if detail_page.select_one('div.title-block__container > div.title-block > div > span') else 'N/A'
        season_episode = detail_page.select_one('div.jw-info-box__container-content > div:nth-child(2) > div:nth-child(2) > h2').text.strip() if detail_page.select_one('div.jw-info-box__container-content > div:nth-child(2) > div:nth-child(2) > h2') else 'N/A'
        genre = [g.text.strip() for g in detail_page.select('div.title-info.visible-xs.visible-sm > div:nth-child(3) > div')] if detail_page.select('div.title-info.visible-xs.visible-sm > div:nth-child(3) > div') else 'N/A'
        summary = detail_page.select_one('div.jw-info-box__container-content > div:nth-child(2) > div:nth-child(6) > p').text.strip() if detail_page.select_one('div.jw-info-box__container-content > div:nth-child(2) > div:nth-child(6) > p') else 'N/A'
        runtime = detail_page.select_one('div.title-info.visible-xs.visible-sm > div:nth-child(4) > div').text.strip() if detail_page.select_one('div.title-info.visible-xs.visible-sm > div:nth-child(4) > div') else 'N/A'
        producer_country = detail_page.select_one('div.title-info.visible-xs.visible-sm > div:nth-child(6) > div').text.strip() if detail_page.select_one('div.title-info.visible-xs.visible-sm > div:nth-child(6) > div') else 'N/A'
        age_rating = detail_page.select_one('div.title-info.visible-xs.visible-sm > div:nth-child(5) > div').text.strip() if detail_page.select_one('div.title-info.visible-xs.visible-sm > div:nth-child(5) > div') else 'N/A'
        director = detail_page.select_one('div.title-info.visible-xs.visible-sm > div:nth-child(7) > div > span > span').text.strip() if detail_page.select_one('div.title-info.visible-xs.visible-sm > div:nth-child(7) > div > span > span') else 'N/A'
        cast_names = [c.text.strip() for c in detail_page.select('div.hidden-horizontal-scrollbar.title-credits div div:nth-child(1) > span')]
        cast_roles = [r.text.strip() for r in detail_page.select('div.hidden-horizontal-scrollbar.title-credits div div:nth-child(1) > div > div > strong')]

        cast = [{'name': name, 'role': role} for name, role in zip(cast_names, cast_roles)]

        
        drama_data.append({
            'title': title,
            'original_title': original_title,
            'year': year,
            'season_episode': season_episode,
            'genre': genre,
            'summary': summary,
            'runtime': runtime,
            'producer_country': producer_country,
            'age_rating': age_rating,
            'director': director,
            'cast': cast
        })
        
        print(f"Scraped: {title},{original_title},{year,season_episode},{genre},{summary},{rating},{runtime},{producer_country}")

finally:
   print('끝')

df = pd.DataFrame(drama_data)
df.tail(2)


Scraped: 안티 히어로,원제: アンチヒーロー,('(2024)', '1개 시즌'),['1시간 6분'],"안티"한 변호사는 정의인가, 악인가――!?

새로운 영웅이 당신의 상식을 뒤집는 역전, 역설 엔터테인먼트, 시동!,N/A,15+,Kenta Tanaka ,  Yohei Miyazaki
Scraped: 삼체,원제: 3 Body Problem,('(2024)', '1개 시즌'),['SF, 스릴러, 드라마, 판타지'],여러 시대에 걸쳐 온 지구를 뒤흔들 사실을 발견한 각 대륙의 저명 과학자 다섯 명. 이제 과학의 법칙들이 밝혀지고 실존적 위협이 드리우기 시작한다.,N/A,56min,미국
Scraped: Ojingeo Geim,원제: Squid Game,('(2021)', '2개 시즌'),['액션, 드라마, 스릴러'],빚에 쫓기는 수백 명의 사람들이 서바이벌 게임에 뛰어든다. 거액의 상금으로 새로운 삶을 시작하기 위해. 하지만 모두 승자가 될 순 없는 법. 탈락하는 이들은 치명적인 결과를 각오해야 한다.,N/A,55min,대한민국
Scraped: 히어로는 아닙니다만,N/A,('(2024)', '1개 시즌'),['SF, 드라마, 액션, 판타지, 로맨스'],현대인의 질병에 걸려 능력을 잃어버린 초능력 가족의 이야기로, 남다른 능력을 지녔지만 아무도 구하지 못했던 남자 복귀주가 마침내 운명의 여자 도다해를 구해내는 판타지 힐링 로맨스,N/A,1시간 12분,대한민국
Scraped: The 8 Show,N/A,('(2024)', '1개 시즌'),['드라마, 스릴러, 코미디'],8인의 남녀가 8층으로 나뉜 비밀스러운 공간에 갇힌다. 그리고 ‘시간이 쌓이면 돈을 버는’ 쇼가 시작된다. 협력과 대립, 반목과 배신을 거듭하는 참가자들. 그들 사이에서 원초적 욕망이 격돌한다.,N/A,52min,대한민국
Scraped: 비밀은 없어,N/A,('(2024)', '1개 시즌'),['코미디, 드라마, 로맨스'],통제 불능 혓바닥 헐크가 된 아나운서 송기백이 열정 충만 예능작가 온우주를 

,title,original_title,year,season_episode,genre,summary,runtime,producer_country,age_rating,director,cast
1348,아스코마치: 아스카 공업 고교 이야기,원제: アスコーマーチ 〜明日香工業高校物語〜,(2011),1개 시즌,[드라마],1지망이었던 여고 입학 실패는 참담한 결과를 가져왔다. 남학생이 득실대는 공고 입학...,45min,일본,15+,Hibino Akira,"[{'name': 'Emi Takei', 'role': 'Nao Yoshino'}]"
1349,El Secreto de la Familia Greco,N/A,(2022),1개 시즌,[47min],겉보기에는 번듯한 가족. 그러나 그들은 높은 생활 수준과 사회적 체면을 유지하고자 ...,19,N/A,멕시코,N/A,[]


In [59]:
df.tail(2)

,title,original_title,year,season_episode,genre,summary,runtime,producer_country,age_rating,director,cast
398,세븐시즈,원제: 7SEEDS,(2019),2개 시즌,"[SF, 액션, 드라마, 공포, 로맨스, 애니메이션]","인류가 사라진 지구, 내성적인 소녀 나츠가 잠에서 깨어난다. 소수의 생존자로 선택받...",24min,N/A,일본,N/A,"[{'name': 'Nao Toyama', 'role': 'Natsu Iwashim..."
399,파이어플라이 레인,원제: Firefly Lane,(2021),2개 시즌,"[드라마, 로맨스]",어릴 적부터 삶의 모든 순간을 함께해온 두 여자. 숱한 고난을 견딘 그들의 우정에 ...,52min,N/A,미국,N/A,"[{'name': 'Katherine Heigl', 'role': 'Tully'}]"


In [62]:
df.to_csv('Justwatch 드라마 크롤링.csv', encoding='utf-8')

In [66]:
df.to_excel('Justwatch 드라마 크롤링.xlsx')

---

# Disney 크롤링

In [67]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import time
import pandas as pd

# 크롬 드라이버 설정 및 페이지 열기
driver = webdriver.Chrome()
driver.get('https://www.justwatch.com/kr/%EB%8F%99%EC%98%81%EC%83%81%EC%84%9C%EB%B9%84%EC%8A%A4/disney-plus/TV-%ED%94%84%EB%A1%9C%EA%B7%B8%EB%9E%A8?genres=drm')

# 페이지 로딩 대기 및 스크롤 끝까지 내리기
try:
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.title-list')))

    SCROLL_PAUSE_TIME = 2
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # 페이지 소스 가져오기
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    
    # 각 드라마의 링크 추출
    drama_links = [a['href'] for a in soup.select('a.title-list-grid__item--link')]
    
    drama_data = []
    
    for link in drama_links:
        driver.get(f"https://www.justwatch.com{link}")
        time.sleep(2)  # 페이지 로딩 대기
        
        detail_page = BeautifulSoup(driver.page_source, 'html.parser')
        
        title = detail_page.select_one('div.title-block h1').text.strip() if detail_page.select_one('div.title-block h1') else 'N/A'
        original_title = detail_page.select_one('div.title-block > h3').text.strip() if detail_page.select_one('div.title-block > h3') else 'N/A'
        year = detail_page.select_one('div.title-block__container > div.title-block > div > span').text.strip() if detail_page.select_one('div.title-block__container > div.title-block > div > span') else 'N/A'
        season_episode = detail_page.select_one('div.jw-info-box__container-content > div:nth-child(2) > div:nth-child(2) > h2').text.strip() if detail_page.select_one('div.jw-info-box__container-content > div:nth-child(2) > div:nth-child(2) > h2') else 'N/A'
        genre = [g.text.strip() for g in detail_page.select('div.title-info.visible-xs.visible-sm > div:nth-child(3) > div')] if detail_page.select('div.title-info.visible-xs.visible-sm > div:nth-child(3) > div') else 'N/A'
        summary = detail_page.select_one('div.jw-info-box__container-content > div:nth-child(2) > div:nth-child(6) > p').text.strip() if detail_page.select_one('div.jw-info-box__container-content > div:nth-child(2) > div:nth-child(6) > p') else 'N/A'
        runtime = detail_page.select_one('div.title-info.visible-xs.visible-sm > div:nth-child(4) > div').text.strip() if detail_page.select_one('div.title-info.visible-xs.visible-sm > div:nth-child(4) > div') else 'N/A'
        producer_country = detail_page.select_one('div.title-info.visible-xs.visible-sm > div:nth-child(6) > div').text.strip() if detail_page.select_one('div.title-info.visible-xs.visible-sm > div:nth-child(6) > div') else 'N/A'
        age_rating = detail_page.select_one('div.title-info.visible-xs.visible-sm > div:nth-child(5) > div').text.strip() if detail_page.select_one('div.title-info.visible-xs.visible-sm > div:nth-child(5) > div') else 'N/A'
        director = detail_page.select_one('div.title-info.visible-xs.visible-sm > div:nth-child(7) > div > span > span').text.strip() if detail_page.select_one('div.title-info.visible-xs.visible-sm > div:nth-child(7) > div > span > span') else 'N/A'
        cast_names = [c.text.strip() for c in detail_page.select('div.hidden-horizontal-scrollbar.title-credits div div:nth-child(1) > span')]
        cast_roles = [r.text.strip() for r in detail_page.select('div.hidden-horizontal-scrollbar.title-credits div div:nth-child(1) > div > div > strong')]

        cast = [{'name': name, 'role': role} for name, role in zip(cast_names, cast_roles)]

        
        drama_data.append({
            'title': title,
            'original_title': original_title,
            'year': year,
            'season_episode': season_episode,
            'genre': genre,
            'summary': summary,
            'runtime': runtime,
            'producer_country': producer_country,
            'age_rating': age_rating,
            'director': director,
            'cast': cast
        })
        
        print(f"Scraped: {title},{original_title},{year,season_episode},{genre},{summary},{rating},{runtime},{producer_country}")

finally:
   print('끝')

df = pd.DataFrame(drama_data)
df.tail(2)


Scraped: 쇼군,원제: Shōgun,('(2024)', '1개 시즌'),['전쟁, 드라마, 역사'],1600년 일본. 요시이 토라나가 군주가 섭정 평의회의 공세에 맞서 사활을 건 전투를 벌이던 어느 날, 근처 어촌에 의문의 유럽 선박이 좌초한다.,N/A,59min,미국
Scraped: 삼식이 삼촌,N/A,('(2024)', '1개 시즌'),['드라마, 역사'],'전쟁 중에도 하루 세끼를 먹였다'는 삼식이 삼촌과 초엘리트 김산이란 두 가상의 인물이 사랑과 믿음 혹은 의심의 이야기를 써나가는 작품,N/A,43min,대한민국
Scraped: 더 베일,원제: The Veil,('(2024)', '1개 시즌'),['드라마, 스릴러'],그녀의 비밀을 알아내는 게 내 임무야. 이스탄불에서 파리, 런던까지 두 여자의 숨 막히는 진실게임이 시작된다.,N/A,46min,미국
Scraped: 아메리칸 호러 스토리,원제: American Horror Story,('(2011)', '12개 시즌'),['드라마, 스릴러, SF, 공포'],'공포'라는 주제로 시즌마다 다른 이야기를 풀어가는 앤솔로지 시리즈. 인간이 가지고 있는 사악한 근성을 탐구하는 동시에 인간이 느끼는 초자연적 두려움과 일상 속의 공포를 조명한다.,N/A,44min,미국
Scraped: Crash,N/A,('(2024)', '1개 시즌'),['액션, 코미디, 범죄, 드라마'],카이스트 출신의 합리적 개인주의자 차연호와 냉철한 판단력과 따뜻한 감수성을 겸비한 교통조사계 에이스 민소희가 교통범죄수사팀에서 펼치는 좌충우돌 도로 위 범죄 추적기,N/A,1시간 2분,대한민국
Scraped: Susabanjang 1958,원제: Chief Detective 1958,('(2024)', '1개 시즌'),['드라마, 범죄, 스릴러'],1958년을 배경으로 야만의 시대, 소도둑 검거 전문 박영한 형사가 개성 넘치는 동료 3인방과 한 팀으로 뭉쳐 부패 권력의 비상식을 상식으로 깨부수며 민중을 위한 형사로 거듭나는 이야기를 그린 드

,title,original_title,year,season_episode,genre,summary,runtime,producer_country,age_rating,director,cast
329,우아한 친구들,N/A,(2020),1개 시즌,"[드라마, 범죄]",40대 부부들이 모여 사는 신도시에 살인사건이 일어난 후 평화로운 일상에 균열이 생...,1시간 7분,대한민국,19,Song Hyun-wook,"[{'name': 'Yu Jun-sang', 'role': 'Ahn Goong-Ch..."
330,Dead or Alive,N/A,(2015),1개 시즌,[43min],What does it feel like to be stranded in the A...,미국,N/A,N/A,N/A,"[{'name': 'Glenn Wrage', 'role': 'Narrator (vo..."


In [68]:
df.to_csv('Justwatch_Disney드라마 크롤링.csv', encoding='utf-8')

In [70]:
df.to_excel('Justwatch_Disney드라마 크롤링.xlsx')